In [1]:
//Janvier : V, #Frevrier: X Zlib, #Mars :X Zlib , Avril : X, Mai:X, juin, juillet

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1611268551592_0001,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
// Imports
import scala.sys.process._
import java.net.URL
import java.io.File
import java.io.File
import java.nio.file.{Files, StandardCopyOption}
import java.net.HttpURLConnection 
import org.apache.spark.sql.functions._

//import scala.sqlContext.implicits._
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.{Row, SparkSession}
import org.apache.spark.sql.functions.{col, to_date, to_timestamp}
import org.apache.spark.sql.types.DateType
import java.util.zip.ZipInputStream
import java.io.BufferedReader
import java.io.InputStreamReader
import org.apache.spark.sql.SQLContext
import com.amazonaws.services.s3.AmazonS3Client
import org.apache.spark.sql.functions.{from_unixtime, unix_timestamp, _}
import org.apache.spark.sql.types.{IntegerType, StringType, StructType}
import org.apache.spark.sql.SaveMode

import org.apache.spark.input.PortableDataStream
import com.amazonaws.auth.BasicAWSCredentials

//import org.apache.spark.sql.cassandra._

//import com.datastax.spark.connector._
import org.apache.spark.sql.types.IntegerType


import com.amazonaws.services.s3.{AmazonS3, AmazonS3ClientBuilder}


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import scala.sys.process._
import java.net.URL
import java.io.File
import java.io.File
import java.nio.file.{Files, StandardCopyOption}
import java.net.HttpURLConnection
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.{Row, SparkSession}
import org.apache.spark.sql.functions.{col, to_date, to_timestamp}
import org.apache.spark.sql.types.DateType
import java.util.zip.ZipInputStream
import java.io.BufferedReader
import java.io.InputStreamReader
import org.apache.spark.sql.SQLContext
import com.amazonaws.services.s3.AmazonS3Client
import org.apache.spark.sql.functions.{from_unixtime, unix_timestamp, _}
import org.apache.spark.sql.types.{IntegerType, StringType, StructType}
import org.apache.spark.sql.SaveMode
import org.apache.spark.input.PortableDataStream
import com.amazonaws.auth.BasicAWSCredentials
import org.apache.spark.sql.types.IntegerType
import com.amazonaws.services.s3.{AmazonS3, AmazonS3ClientBuilder}


In [1]:
// English Data
val list_csv_translate = spark.read.format("csv").option("delimiter", " ").
    csv("s3://testfuret/masterfilelist_translation.txt").   
    withColumnRenamed("_c0","size").
    withColumnRenamed("_c1","hash").
    withColumnRenamed("_c2","url")


val list_2020_translate_tot = list_csv_translate.where(col("url").like("%202002%.gkg.csv.zip"))   
list_2020_translate_tot.show(false)

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
9,application_1611268551592_0010,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

list_csv_translate: org.apache.spark.sql.DataFrame = [size: string, hash: string ... 1 more field]
list_2020_translate_tot: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [size: string, hash: string ... 1 more field]
+-------+--------------------------------+---------------------------------------------------------------------------+
|size   |hash                            |url                                                                        |
+-------+--------------------------------+---------------------------------------------------------------------------+
|6178636|72d0e5ca10362beca19a0510854f4721|http://data.gdeltproject.org/gdeltv2/20200201000000.translation.gkg.csv.zip|
|6133442|765b9d4a4dd3270732defd66de74be37|http://data.gdeltproject.org/gdeltv2/20200201001500.translation.gkg.csv.zip|
|6454334|209299355a96b7e5a9ecf3ec77d2f6de|http://data.gdeltproject.org/gdeltv2/20200201003000.translation.gkg.csv.zip|
|5654513|1ddd9159fce3a3ebf01be064bdbc4bfa|http://data.gdelt

In [4]:
val gkgRDD = sc.binaryFiles("s3://testfuret/Master_file /*202002*.gkg.csv.zip",100).    
   flatMap {  
       
       // decompresser les fichiers
       case (name: String, content: PortableDataStream) =>

          val zis = new ZipInputStream(content.open)
          Stream.continually(zis.getNextEntry).
                //takeWhile(_ != null).
                takeWhile{
                      case null => zis.closeEntry(); zis.close(); false
                      case _ => true
                     }.
                flatMap { _ =>
                   
                    val br = new BufferedReader(new InputStreamReader(zis))
                    Stream.continually(br.readLine()).takeWhile(_ != null)  
                    
                }
         

    }

val gkgDF = gkgRDD.map(x => x.split("\t")).toDF()//.map(row => row.mkString(";")).map(x => x.split(";")).toDF()



val dfGKG = gkgDF.withColumn("_tmp", $"value").select(
    $"_tmp".getItem(0).as("GKGRECORDID"),
    $"_tmp".getItem(1).as("V2DATE"),
    //$"_tmp".getItem(2).as("V2SOURCECOLLECTIONIDENTIFIER"),
    $"_tmp".getItem(3).as("V2SOURCECOMMONNAME"),
    $"_tmp".getItem(4).as("V2DOCUMENTIDENTIFIER"),
    //$"_tmp".getItem(5).as("V1COUNTS"),
    //$"_tmp".getItem(6).as("V2.1COUNTS"),
    $"_tmp".getItem(7).as("V1THEMES"),
    //$"_tmp".getItem(8).as("V2ENHANCEDTHEMES"),
    $"_tmp".getItem(9).as("V1LOCATIONS"),
    //$"_tmp".getItem(10).as("V2ENHANCEDLOCATIONS"),
    $"_tmp".getItem(11).as("V1PERSONS"),
    //$"_tmp".getItem(12).as("V2ENHANCEDPERSONS"),
    //$"_tmp".getItem(13).as("V1ORGANIZATIONS"),
    //$"_tmp".getItem(14).as("V2ENHANCEDORGANIZATIONS"),
    $"_tmp".getItem(15).as("V1TONE")
    //$"_tmp".getItem(16).as("V2.1ENHANCEDDATES"),
    //$"_tmp".getItem(17).as("V2GCAM"),
    //$"_tmp".getItem(18).as("V21SHARINGIMAGE"),
    //$"_tmp".getItem(19).as("V21RELATEDIMAGES"),
    //$"_tmp".getItem(20).as("V21SOCIALIMAGEEMBEDS"),
    //$"_tmp".getItem(21).as("V21SOCIALVIDEOEMBEDS"),
    //$"_tmp".getItem(22).as("V21QUOTATIONS"),
    //$"_tmp".getItem(23).as("V21ALLNAMES"),
    //$"_tmp".getItem(24).as("V21AMOUNTS"),
    //$"_tmp".getItem(25).as("V21TRANSLATIONINFO"),
    //$"_tmp".getItem(26).as("V2EXTRASXML")
    )

//dfGKG.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

gkgRDD: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[15] at flatMap at <console>:57
gkgDF: org.apache.spark.sql.DataFrame = [value: array<string>]
dfGKG: org.apache.spark.sql.DataFrame = [GKGRECORDID: string, V2DATE: string ... 6 more fields]


In [4]:
dfGKG.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------+------------------+--------------------+--------+-----------+---------+------+
|GKGRECORDID|  V2DATE|V2SOURCECOMMONNAME|V2DOCUMENTIDENTIFIER|V1THEMES|V1LOCATIONS|V1PERSONS|V1TONE|
+-----------+--------+------------------+--------------------+--------+-----------+---------+------+
|  896047517|20190101|              2019|           2019.0027|        |           |         |   USA|
|  896047518|20190101|              2019|           2019.0027|     CAN|           |         |   GBR|
|  896047519|20190101|              2019|           2019.0027|        |           |         |   GBR|
|  896047520|20190101|              2019|           2019.0027|     CYP|           |         |   GBR|
|  896047521|20190101|              2019|           2019.0027|     ETH|           |         |      |
|  896047522|20190101|              2019|           2019.0027|     ETH|           |         |      |
|  896047523|20190101|              2019|           2019.0027|     GBR|           |        

In [6]:
val mentionRDD = sc.binaryFiles("s3://testfuret/Master_file /*202002*.mentions*",100000).    
   flatMap {  
       
       // decompresser les fichiers
       case (name: String, content: PortableDataStream) =>

          val zis = new ZipInputStream(content.open)
          Stream.continually(zis.getNextEntry).
                //takeWhile(_ != null).
                takeWhile{
                      case null => zis.closeEntry(); zis.close(); false
                      case _ => true
                     }.
                flatMap { _ =>
                   
                    val br = new BufferedReader(new InputStreamReader(zis))
                    Stream.continually(br.readLine()).takeWhile(_ != null)  
                    
                }
    }




val mentionDF = mentionRDD.map(x => x.split("\t")).map(row => row.mkString(";")).map(x => x.split(";")).toDF()



val dfMention = mentionDF.withColumn("_tmp", $"value").select(
     $"_tmp".getItem(0).as("globaleventID"),
     //$"_tmp".getItem(1).as("EventTimeDate"),
     //$"_tmp".getItem(2).as("MentionTimeDate"),
     //$"_tmp".getItem(3).as("MentionType"),
     $"_tmp".getItem(4).as("MentionSourceName"),
     $"_tmp".getItem(5).as("MentionIdentifier"),
     //$"_tmp".getItem(6).as("SentenceID"),
     //$"_tmp".getItem(7).as("Actor1CharOffset"),
     //$"_tmp".getItem(8).as("Actor2CharOffset"),
     //$"_tmp".getItem(9).as("ActionCharOffset"),
     //$"_tmp".getItem(10).as("InRawText"),
     //$"_tmp".getItem(11).as("Confidence"),
     //$"_tmp".getItem(12).as("MentionDocLen"),
     //$"_tmp".getItem(13).as("MentionDocTone"),
     $"_tmp".getItem(14).as("MentionDocTranslationInfo")
     //$"_tmp".getItem(15).as("Extras")
     )

//dfMention.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

mentionRDD: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[23] at flatMap at <console>:57
mentionDF: org.apache.spark.sql.DataFrame = [value: array<string>]
dfMention: org.apache.spark.sql.DataFrame = [globaleventID: string, MentionSourceName: string ... 2 more fields]


In [7]:
val EventRDD = sc.binaryFiles("s3://testfuret/Master_file /*202002*.export*",100000).    
   flatMap {  
       
       // decompresser les fichiers
       case (name: String, content: PortableDataStream) =>

          val zis = new ZipInputStream(content.open)
          Stream.continually(zis.getNextEntry).
                //takeWhile(_ != null).
                takeWhile{
                      case null => zis.closeEntry(); zis.close(); false
                      case _ => true
                     }.
                flatMap { _ =>
                   
                    val br = new BufferedReader(new InputStreamReader(zis))
                    Stream.continually(br.readLine()).takeWhile(_ != null)  
                    
                }
        
           

    }





val eventDF = EventRDD.map(x => x.split("\t")).map(row => row.mkString(";")).map(x => x.split(";")).toDF()


val dfEvent = eventDF.withColumn("_tmp", $"value").select(
     $"_tmp".getItem(0).as("globaleventID"),
     $"_tmp".getItem(1).as("day"),
     $"_tmp".getItem(2).as("month"),
     $"_tmp".getItem(3).as("year"),
     //$"_tmp".getItem(4).as("fractiondate"),
     //$"_tmp".getItem(5).as("Actor1code"),
     //$"_tmp".getItem(6).as("Actor1Name"),
     //$"_tmp".getItem(7).as("Actor1ContryCode"),
     //$"_tmp".getItem(8).as("actor1KnownGroupeCode"),
     //$"_tmp".getItem(9).as("Actor1EthnicCode"),
     //$"_tmp".getItem(10).as("Actor1Religion1Code"),
     //$"_tmp".getItem(11).as("Actor1Religion2code"),
     //$"_tmp".getItem(12).as("Actor1type1Code"),
     //$"_tmp".getItem(13).as("Actor1Type2code"),
     //$"_tmp".getItem(14).as("Actor1Type3code"),
     //$"_tmp".getItem(15).as("Actor2code"),
     //$"_tmp".getItem(16).as("Actor2Name"),
     //$"_tmp".getItem(17).as("Actor2ContryCode"),
     //$"_tmp".getItem(18).as("Actor2KnownGroupeCode"),
     //$"_tmp".getItem(19).as("Actor2EthnicCode"),
     //$"_tmp".getItem(20).as("Actor2Religion1Code"),
     //$"_tmp".getItem(21).as("Actor2Religion2code"),
     //$"_tmp".getItem(22).as("Actor2type1Code"),
     //$"_tmp".getItem(23).as("Actor2Type2code"),
     //$"_tmp".getItem(24).as("Actor2Type3code"),
     //$"_tmp".getItem(25).as("IsRootEvent"),
     //$"_tmp".getItem(26).as("EventCode"),
     //$"_tmp".getItem(27).as("EventBaseCode"),
     //$"_tmp".getItem(28).as("EventRootCode"),
     //$"_tmp".getItem(29).as("QuadClass"),
     //$"_tmp".getItem(30).as("GoldsteinScale"),
     $"_tmp".getItem(31).as("NumMentions"),
     $"_tmp".getItem(32).as("NumSources"),
     $"_tmp".getItem(33).as("NumArticles"),
     $"_tmp".getItem(34).as("AvgTone"),
     //$"_tmp".getItem(35).as("Actor1Geo_Type"),
     //$"_tmp".getItem(36).as("Actor1Geo_Fullname"),
     //$"_tmp".getItem(37).as("Actor1Geo_CountryCode"),
     //$"_tmp".getItem(38).as("Actor1Geo_ADM1Code"),
     //$"_tmp".getItem(39).as("Actor1Geo_ADM2Code"),
     //$"_tmp".getItem(40).as("Actor1Geo_Lat"),
     //$"_tmp".getItem(41).as("Actor1Geo_Long"),
     //$"_tmp".getItem(42).as("Actor1Geo_FeatureID"),
     //$"_tmp".getItem(43).as("Actor2Geo_Type"),
     //$"_tmp".getItem(44).as("Actor2Geo_Fullname"),
     //$"_tmp".getItem(45).as("Actor2Geo_CountryCode"),
     //$"_tmp".getItem(46).as("Actor2Geo_ADM1Code"),
     //$"_tmp".getItem(47).as("Actor2Geo_ADM2Code"),
     //$"_tmp".getItem(48).as("Actor2Geo_Lat"),
     //$"_tmp".getItem(49).as("Actor2Geo_Long"),
     //$"_tmp".getItem(50).as("Actor2Geo_FeatureID"),
     //$"_tmp".getItem(51).as("ActionGeo_Type"),
     $"_tmp".getItem(52).as("ActionGeo_Fullname")
     //$"_tmp".getItem(53).as("ActionGeo_CountryCode"),
     //$"_tmp".getItem(54).as("ActionGeo_ADM1Code"),
     //$"_tmp".getItem(55).as("ActionGeo_ADM2Code"),
     //$"_tmp".getItem(56).as("ActionGeo_Lat"),
     //$"_tmp".getItem(57).as("ActionGeo_Long"),
     //$"_tmp".getItem(58).as("ActionGeo_FeatureID"),
     //$"_tmp".getItem(59).as("DATEADDED")
    )


//dfEvent.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

EventRDD: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[28] at flatMap at <console>:57
eventDF: org.apache.spark.sql.DataFrame = [value: array<string>]
dfEvent: org.apache.spark.sql.DataFrame = [globaleventID: string, day: string ... 7 more fields]


In [ ]:
dfEvent.count()
//Janvier : 4705004, #Frevrier: X, 

In [ ]:
dfMention.count()

val dfGKG_Covid = dfGKG_Pandemic.union(dfGKG_Corona).distinct()

# Fred Maj

In [34]:
dfGKG.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res175: Long = 5088881


In [8]:
val dfGKG_Pand = dfGKG.
        withColumn("CheckCOVID", $"V1THEMES".like("%PANDEMIC%") || $"V1THEMES".like("%CORONAVIRUS%"))

val dfGKG_Pandemic = dfGKG_Pand.
    filter((dfGKG_Pand("CheckCOVID") === "true"))
//dfGKG_Pandemic.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dfGKG_Pand: org.apache.spark.sql.DataFrame = [GKGRECORDID: string, V2DATE: string ... 7 more fields]
dfGKG_Pandemic: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [GKGRECORDID: string, V2DATE: string ... 7 more fields]


In [28]:
dfGKG_Pandemic.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res92: Long = 457035


In [9]:
val df_gkg_mention_corona =dfMention.join(
  dfGKG_Pandemic,
  dfMention("MentionIdentifier") <=> dfGKG_Pandemic("V2DOCUMENTIDENTIFIER")
  )


// val df_gkg_mention_corona =dfMention.join(
//   broadcast(dfGKG_Pandemic),
//   dfMention("MentionIdentifier") <=> dfGKG_Pandemic("V2DOCUMENTIDENTIFIER")
//   )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df_gkg_mention_corona: org.apache.spark.sql.DataFrame = [globaleventID: string, MentionSourceName: string ... 11 more fields]


In [ ]:
dfMention.count()

In [48]:
df_gkg_mention_corona.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res229: Long = 1011840


In [10]:
//val join_Envent_Mention_Gkg_corrona = dfMention.join(dfEvent, dfMention("globaleventID")===dfEvent("globaleventID"), "inner").

val join_Envent_Mention_Gkg_corrona =dfEvent.join(
  df_gkg_mention_corona,
  dfEvent("globaleventID") <=> df_gkg_mention_corona("globaleventID")
  ).drop(dfEvent("globaleventID"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

join_Envent_Mention_Gkg_corrona: org.apache.spark.sql.DataFrame = [day: string, month: string ... 19 more fields]


In [8]:
join_Envent_Mention_Gkg_corrona.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+----+-----------+----------+-----------+-----------------+--------------------+-------------+--------------------+--------------------+-------------------------+-------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|     day| month|year|NumMentions|NumSources|NumArticles|          AvgTone|  ActionGeo_Fullname|globaleventID|   MentionSourceName|   MentionIdentifier|MentionDocTranslationInfo|        GKGRECORDID|        V2DATE|  V2SOURCECOMMONNAME|V2DOCUMENTIDENTIFIER|            V1THEMES|         V1LOCATIONS|           V1PERSONS|              V1TONE|CheckCOVID|
+--------+------+----+-----------+----------+-----------+-----------------+--------------------+-------------+--------------------+--------------------+-------------------------+-------------------+--------------+--------------------+--------------------+--------------------+--------------------+-

In [12]:
//join_Envent_Mention_Gkg_corrona.write.mode('append').json("s3://testfuret/result/Janvier_CSV/")
join_Envent_Mention_Gkg_corrona.write.format("org.apache.spark.sql.json").mode(SaveMode.Append).save("s3://cgl2/fevrier/");

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
org.apache.spark.SparkException: Job aborted.
  at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
  at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:174)
  at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
  at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
  at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
  at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:173)
  at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:169)
  at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:197)
  at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
  at org.apache.

In [9]:

//Save In Parquet
join_Envent_Mention_Gkg_corrona.write.parquet("s3://testfuret/result/fevrier")
//val df = spark.
//       read.
//       parquet("même chemin")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

KeyboardInterrupt: 

In [ ]:
//dfGKG_Covid.write.mode(SaveMode.Overwrite).format("parquet").saveAsTable("s3://testfuret/result/dfGKG_Corona.parquet")
//dfGKG_Covid.write.mode.save("s3://testfuret/result/dfGKG_Corona.parquet", SaveMode.Append )
dfGKG_Covid.write.format("parquet").mode("append").save("dfGKG_Covid.parquet")

In [16]:
//f_gkg_mention_corona.format("parquet").mode("append").save("dfGKG_Covid.parquet")

df_gkg_mention_corona.write.parquet("s3://testfuret/result/df_gkg_mention_corona.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
org.apache.spark.SparkException: Job aborted.
  at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
  at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:174)
  at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
  at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
  at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
  at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:173)
  at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:169)
  at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:197)
  at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
  at org.apache.

In [ ]:
sc.binaryFiles("s3://axel-michalewicz-telecom/masterFileTrans/mentions",100).foreach{r=> {
    val zis = new ZipInputStream(r.open)
}

}

In [ ]:
//dF.write.parquet("s3://testfuret/result/jk/")
//val df = spark.
//       read.
//       parquet("même chemin")